# Workflow automation for a recommender engine using ALS Model in PySpark
### This notebook demonstrates the automation of a product recommendation engine given we have user-item interaction data in terms of the frequency of purchase for each unique user-item pair
### The automation is implemented by deploying the notebook as a job which reads the input data from a remote database, trains and runs the model and writes the output back to the designated database


## Importing the libraries and starting the Spark Session

In [1]:
### Load packages 
import time
t0 = time.time()
import pyspark.sql.functions as sql_func
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS, ALSModel # factorizacion de matrices
from pyspark.context import SparkContext 
from pyspark.sql.session import SparkSession
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics
from pyspark.ml.evaluation import RegressionEvaluator
import jaydebeapi, pandas as pd

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200918213828-0004
KERNEL_ID = 56083c30-7ef9-48aa-a201-9238e70c0f1e


In [2]:
### set up spark session and context 
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## Remote connection to the database ( Any supported database like DB2, Netezza, PDA )
### To enable the connection, add your database under the Data Sources tab in your project. You would need information about your database like JDBC URL, type, username/password

### Adding the asset (for example- the dataset which has the user-item interaction information) from the remote database after setting up the connection

In [3]:
# Access to Porject's conexion
# and get the data 
from project_lib import Project
project = Project.access()
PDA_COPPEL_2020_credentials = project.get_connection(name="PDA_Analytics")

PDA_COPPEL_2020_connection = jaydebeapi.connect('org.netezza.Driver',
    '{}://{}:{}/{}'.format('jdbc:netezza',
    PDA_COPPEL_2020_credentials['host'],
    PDA_COPPEL_2020_credentials['port'],
    PDA_COPPEL_2020_credentials['database']),
    [PDA_COPPEL_2020_credentials['username'],
    PDA_COPPEL_2020_credentials['password']])

query = '''
SELECT CLIENTECODIGO as ID_CTE, CAST( ADCLAFAM as integer) as ID_CLAS1, 
		FRECUENCIA as FREQUENCY  
	FROM(	SELECT *, TRIM(TO_CHAR(FAMILIA,'000')) AS FAM, TRIM(TO_CHAR(CLASE,'00')) AS CLAS, 
	                  TRIM(TO_CHAR(AREA,'0')) AS AREA, (AREA||DEPARTAMENTO||CLAS||FAM) AS ADCLAFAM 
				FROM(    SELECT  CLIENTECODIGO, CLASE, FAMILIA, DEPARTAMENTO, 
				                 SUM(CASE WHEN CLASE>'0' THEN 1 ELSE 0 END) AS FRECUENCIA,
				                 MAX(CASE WHEN CARTERA='Ropa' THEN 1  
								          WHEN CARTERA='Muebles' THEN 2
										  WHEN CARTERA='Prestamos' THEN 3 ELSE 0 END) AS AREA
							FROM( SELECT *, CASE WHEN CLASE>'0' THEN 1 ELSE 0 END AS T_CLASE, 
							               CASE WHEN FAMILIA>'0' THEN 1 ELSE 0 END AS T_FAMILIA
									FROM DIRECCIONRIESGOS.ADMIN.TRANSACCIONESCARTERAS 
									where FECHACORTE between '2017-01-31' and '2019-12-31' 
									and CLIENTECODIGO not in (9001,9000) AND CLIENTECODIGO >10000) E 
				GROUP BY CLIENTECODIGO, CLASE, FAMILIA, DEPARTAMENTO 
				ORDER BY CLIENTECODIGO) T 
	WHERE CLASE NOT IN (-99)) P 
	ORDER BY CLIENTECODIGO,ADCLAFAM limit 318828
'''
# 1% ~ 40 minutos 
# 0.1% ~ 6 minutos 
# medimos el tiempo en transferir los datos 
t1 = time.time()
data_df_1 = pd.read_sql(query, con=PDA_COPPEL_2020_connection)
t2 = time.time()
seconds_get_data = t2 - t1 
print(str(seconds_get_data / 60 ) + ' minutos en transferir tabla')
data_df_1.head(10)

2.9779995759328206 minutos en transferir tabla


,ID_CTE,ID_CLAS1,FREQUENCY
0,10002,1313064,1.0
1,10002,1314064,3.0
2,10002,1862008,4.0
3,10002,1867048,2.0
4,10002,2210070,1.0
5,10002,2222011,1.0
6,10002,2229008,1.0
7,10002,2244001,1.0
8,10002,2248001,1.0
9,10002,2665020,1.0


In [4]:
print(data_df_1.dtypes) # #  validamos el tipo de dato y numero de registros
print(data_df_1.shape)

ID_CTE         int64
ID_CLAS1       int64
FREQUENCY    float64
dtype: object
(318828, 3)


In [5]:
spark_df = spark.createDataFrame(data_df_1) # distribuimos el pandas dataframe a spark 
# hacer el get desde spark 

In [6]:
# spark_df.show() #  no es necesario verlo 

### Preparing data for the model

In [7]:
ratings = (spark_df
    .select(
        'ID_CTE',
        'ID_CLAS1',
        'FREQUENCY',
    )
).cache() #  lo cargamos a memoria ram 

### Make sure your data is 'integer' type 

### Spliting the data set to test and train for measuring the performance of the ALS Model

In [8]:
(training, test) = ratings.randomSplit([0.8, 0.2]) #  debemos validar esto 

## Build the recommendation model using ALS on the training data
### Collaborative filtering is commonly used for recommender systems. These techniques aim to fill in the missing entries of a user-item association matrix. spark.ml currently supports model-based collaborative filtering, in which users and products are described by a small set of latent factors that can be used to predict missing entries. spark.ml uses the alternating least squares (ALS) algorithm to learn these latent factors. 

### Parameters of ALS Model in PySpark realization are following:

##### NumBlocks is the number of blocks the users and items will be partitioned into in order to parallelize computation.
##### rank is the number of latent factors in the model.
##### maxIter is the maximum number of iterations to run.
##### regParam specifies the regularization parameter in ALS.
##### implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data (defaults to false which means using explicit feedback).
##### alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations (defaults to 1.0)

### Explicit vs. implicit feedback
#### The standard approach to matrix factorization based collaborative filtering treats the entries in the user-item matrix as explicit preferences given by the user to the item, for example, users giving ratings to products.
#### It is common in many real-world use cases to only have access to implicit feedback (e.g. views, clicks, purchases, likes, shares etc.). The approach used in spark.ml to deal with such data is taken from Collaborative Filtering for Implicit Feedback Datasets. Essentially, instead of trying to model the matrix of ratings directly, this approach treats the data as numbers representing the strength in observations of user actions (such as the number of clicks). Those numbers are then related to the level of confidence in observed user preferences, rather than explicit ratings given to items. The model then tries to find latent factors that can be used to predict the expected preference of a user for an item.

In [9]:
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
t2 = time.time()
als = ALS(maxIter=10, regParam=0.01, rank = 100, 
          userCol="ID_CTE", itemCol="ID_CLAS1", ratingCol="FREQUENCY",
          coldStartStrategy="drop",
          implicitPrefs=True)

model = als.fit(ratings)

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="FREQUENCY",
                                predictionCol="prediction")
t3 = time.time()
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))
print('Tiempo de entranar y predecir    ' + str( (t3- t2)/60) + '    minutos') 

Root-mean-square error = 3.910787279063556
Tiempo de entranar y predecir    0.7275614261627197    minutos


###  Generate top k Item recommendations for each user
#### The value of 'k' here is 10 and can be changed by passing the desired value to the function



In [10]:
userRecs = model.recommendForAllUsers(10)
userRecs.count()

17407

#### Display the results : Each row represents the 'k' recommendations for each User

In [11]:
#userRecs.take(10)

#### For getting each recommendation as a row in the final csv, we break down the result generated above using the explode function

In [12]:
from pyspark.sql.functions import explode
userRecs1=userRecs.withColumn("recommendations", explode(userRecs.recommendations))
userRecs1.show()

+------+--------------------+
|ID_CTE|     recommendations|
+------+--------------------+
| 18911|[2968001, 0.9675391]|
| 18911|[2920001, 0.7479556]|
| 18911|[2910001, 0.5792166]|
| 18911|[2990003, 0.5360558]|
| 18911|[2990001, 0.52307...|
| 18911|[2975001, 0.47655...|
| 18911|[2950003, 0.22225...|
| 18911|[2901001, 0.20224...|
| 18911|[2920004, 0.18126...|
| 18911|[2901004, 0.15629...|
| 37111|[1318203, 1.0439721]|
| 37111|[1314063, 1.0095295]|
| 37111|[1860048, 0.97131...|
| 37111|[1102016, 0.96551...|
| 37111|[1318003, 0.9465939]|
| 37111|[1862009, 0.9254286]|
| 37111|[1102012, 0.9153261]|
| 37111|[1318073, 0.8597628]|
| 37111|[2121001, 0.8550992]|
| 37111|[1646081, 0.8025979]|
+------+--------------------+
only showing top 20 rows



####  Breaking down reach recommendation to separate columns

In [13]:
#import select as s   PDA_Analytics


In [14]:
userRecs1= userRecs1 \
  .select('ID_CTE', 'recommendations.*')       

### Writing the Output back to the Remote Datasource
#### Thereby the output or resulting csv can be consumed directly by anyone who has the access to the remote database 

In [15]:
'''new_table_name = 'ANALITICAAFORE.ADMIN.RECOMMENDATIONSRESULT_TEST_CENIC'

userRecs1.coalesce(2).write.format('jdbc') \
    .mode('overwrite') \
    .option('url', 'jdbc:netezza://{}:{}/{}'.format(PDA_COPPEL_2020_credentials['host'],PDA_COPPEL_2020_credentials['port'], PDA_COPPEL_2020_credentials['database'])) \
    .option('dbtable', new_table_name) \
    .option('user', PDA_COPPEL_2020_credentials['username']).option('driver','org.netezza.Driver').option('password', PDA_COPPEL_2020_credentials['password']).save()
'''    

"new_table_name = 'ANALITICAAFORE.ADMIN.RECOMMENDATIONSRESULT_TEST_CENIC'\n\nuserRecs1.coalesce(2).write.format('jdbc')     .mode('overwrite')     .option('url', 'jdbc:netezza://{}:{}/{}'.format(PDA_COPPEL_2020_credentials['host'],PDA_COPPEL_2020_credentials['port'], PDA_COPPEL_2020_credentials['database']))     .option('dbtable', new_table_name)     .option('user', PDA_COPPEL_2020_credentials['username']).option('driver','org.netezza.Driver').option('password', PDA_COPPEL_2020_credentials['password']).save()\n"

In [16]:
userRecs1.show(5)

+------+--------+----------+
|ID_CTE|ID_CLAS1|    rating|
+------+--------+----------+
| 18911| 2968001| 0.9675391|
| 18911| 2920001| 0.7479556|
| 18911| 2910001| 0.5792166|
| 18911| 2990003| 0.5360558|
| 18911| 2990001|0.52307117|
+------+--------+----------+
only showing top 5 rows



In [17]:
new_table_name = 'RecommendationsResult_test_cenic'

userRecs1.coalesce(1).write.format('jdbc') \
    .mode('overwrite') \
    .option('url', 'jdbc:netezza://{}:{}/{}'.format(PDA_COPPEL_2020_credentials['host'],PDA_COPPEL_2020_credentials['port'],'CENIC')) \
    .option('dbtable', new_table_name) \
    .option('user', PDA_COPPEL_2020_credentials['username']).option('driver','org.netezza.Driver').option('password', PDA_COPPEL_2020_credentials['password']).save()

## Saving the model for deployment in WML

In [18]:
#!pip install  --proxy=https://10.33.128.80:8080 dsx proxy para usar pip 

### Pack the model inside a pipeline 
#### Since the WML deployments allow saving Spark Pipelines directly, put the ALS model inside a Pipeline for direct deployment stage
#### Typically, A Pipeline is specified as a sequence of stages, and each stage is either a Transformer or an Estimator. These stages are run in order, and the input DataFrame is transformed as it passes through each stage. For this case, since the pipeline is bought in to action just for the sole cause of deployment, we do not use any transformers and such

In [19]:
from pyspark.ml import Pipeline

In [20]:
pipeline = Pipeline(stages=[model])

In [21]:
model_alsWML = pipeline.fit(ratings)

In [22]:
# model_alsWML.save('/temp/')

In [23]:
t_final = time.time()

In [24]:
print(t_final- t0)

369.10957765579224


In [25]:
print((t_final- t0) / 60 )

6.151826294263204
